# <span id="chap_parallel_ipython"></span> Native IPython parallelism

IPython has some useful parallel programming constructs built-in. They're so useful and so integrated into the notebook interface that they make a perfect place to start when looking to speed-up simulations &ndash; and indeed are a perfect place to stop, too, for the most applications.

## <span id="sec_ipython_parallel_architecture"></span>The IPython parallel computing architecture

IPython comes with a parallel computing library built-in. This library is very general and provides for several different parallel processing "idioms", but there's one that's especially useful for network science simulations so we'll focus exclusively on that. If you ever need something more flexible, or want to understand parallel computing more generally, there are plenty of material available: a good on-line tutorial is [[GPRK11]](bibliography.ipynb#GPRK11), while [[Ros13]](bibliography.ipynb#Ros13) also covers the material in some depth.

When we talk about a "computing architecture" we mean the components that go together to form the computing solution: the "mental map" a programmer needs in order to use the system. A computing architecture consists of *physical* and *logical* elements. The physical elements are a set of one or more machines connected by one of more interconnects: I'll use the term "interconnect" to refer to communication links between machines, instead of the more usual term "network", for reasons that are hopefully obvious! Typically we have a number of machines being used for computing, and we'll refer to these machines as a *cluster*. Each machine in the cluster will have one or more cores that can be used for computing. Within the cluster there is a single machine that manages the compute services being provided, and we'll refer to this machine as the *cluster head*. The other machines are dedicated to providing compute services, and we refer to these as *workers*. Outside the cluster there will be one or more *client* machines being used by scientists, which may be deaktop or laptop machines, and may be powerful in their own right.

The logical elements are a set of processes, each one running on one of the physical elements. Each client will have one or more IPython processes to manage the interface with the users, and we'll refer to these processes generically as *notebooks* even though they might actually be just scripts. On the cluster head there is a single *controller* that acts as the gateway to the cluster's compute resources. On the cluster workers there are a number of *engines* that connect to the controller. How many engines? Typically one engine per core, although in some circumstances (such as when you're sharing your cluster with ither users) you might start fewer. (There's never any particularly good reason to start more engines on a machine than it has cores.)

Each process in the logical architecture is its own IPython process, and that's important. The data structures and functions you create in your notebook aren't automatically available to the engines. You have to import or communicate code to the engines in order for them to run it; you have to store data in files, or pass it over the interconnect, in order for them to have something to process. These difference can be fiddly, and we'll deal with them in some depth below.

To recap, the notebook views the cluster through the controller on the cluster head, which is connected to engines running on cluster workers, with there usually being as many engines available as the cluster workers have cores. The idea is that work generated in the notebook will be executed by the engines, and there are enough engines to soak-up all the computing resources available in the cluster. 

<div class=figure id=fig_ipython_architecture>
<div class=figurebody>
<img alt="IPython parallel computing architecture" src="ipython-parallelism.svg">
<br>
<span class=caption>The IPython parallel architecture. Processes map to different machines. Multicore workers will have several engines</span>
</div>
</div>